In [1]:
product_review <- readr::read_csv("../input/toyamazonPhilips.csv")
dim(product_review)
library(pander)
pandoc.table(product_review[2:4,1:3], 
             justify = c('left', 'left', 'center'), style = 'grid')

Parsed with column specification:
cols(
  name = col_character(),
  review = col_character(),
  rating = col_double(),
  count = col_double(),
  standev = col_double()
)



[1] 191   5



+--------------------------+--------------------------------+--------+
| name                     | review                         | rating |
+==========================+================================+========+
| Philips Avent 3 Pack 9oz | If I had not been given a ton  |   2    |
| Bottles                  | of Avent bottles, I would have |        |
|                          | chosen some other system.  The |        |
|                          | leaking is terrible!!!  You    |        |
|                          | have to buy the disks          |        |
|                          | separately, you should get     |        |
|                          | them for free because they are |        |
|                          | absolutely essential.  The     |        |
|                          | only way to mix formula in the |        |
|                          | bottle or transport liquid is  |        |
|                          | to use the disks in the ring,  |        |
|   

In [2]:
table(product_review$rating)


 1  2  3  4  5 
45 33 17 30 66 

In [3]:
library(dplyr)
product_review <- product_review %>% filter(rating != 3) %>% 
                    mutate(rating_new = if_else(rating >= 4, 1, 0))

product_review_training <-  product_review[1:150, ]



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [4]:
library(tm)
corpus_toy <- Corpus(VectorSource(product_review_training$review))
tdm_toy <- DocumentTermMatrix(corpus_toy, list(removePunctuation = TRUE, 
                                               removeNumbers = TRUE))

training_set_toy <- as.matrix(tdm_toy)

training_set_toy <- cbind(training_set_toy, product_review_training$rating_new)

colnames(training_set_toy)[ncol(training_set_toy)] <- "y"

training_set_toy <- as.data.frame(training_set_toy)
training_set_toy$y <- as.factor(training_set_toy$y)

Loading required package: NLP


Attaching package: ‘NLP’


The following object is masked from ‘package:httr’:

    content




In [5]:
library(caret)
review_toy_model <- train(y ~., data = training_set_toy, method = 'svmLinear3')

Loading required package: lattice

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:NLP’:

    annotate



Attaching package: ‘caret’


The following object is masked from ‘package:httr’:

    progress




In [6]:
test_review_data <- product_review[151:191, ]

test_corpus <- Corpus(VectorSource(test_review_data$review))
test_tdm <- DocumentTermMatrix(test_corpus, control=list(dictionary = Terms(tdm_toy)))
test_tdm <- as.matrix(test_tdm)

#Build the prediction  
model_toy_result <- predict(review_toy_model, newdata = test_tdm)

check_accuracy <- as.data.frame(cbind(prediction = model_toy_result, rating = test_review_data$rating_new))

check_accuracy <- check_accuracy %>% mutate(prediction = as.integer(prediction) - 1)

check_accuracy$accuracy <- if_else(check_accuracy$prediction == check_accuracy$rating, 1, 0)
round(prop.table(table(check_accuracy$accuracy)), 3)


    0     1 
0.208 0.792 